# Experimenting with assistants

## Importing relevant modules

In [16]:
from openai import OpenAI
import shelve
from dotenv import load_dotenv
import os
import time

## Setup a client

In [17]:
load_dotenv()
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
client = OpenAI(api_key=OPEN_AI_API_KEY)

## Upload file

In [18]:
def upload_file(path):
    # Upload a file with an "assistants" purpose
    file = client.files.create(file=open(path, "rb"), purpose="assistants")
    return file


In [19]:
file = upload_file("../docs/airbnb-faq.pdf")
file

FileObject(id='file-HQg2T0R5AGQgDFRryfXmH2AT', bytes=64571, created_at=1730382491, filename='airbnb-faq.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [20]:
file.id

'file-HQg2T0R5AGQgDFRryfXmH2AT'

## Create assistant

In [68]:
client.beta.assistants.create(
        name="WhatsApp AirBnb Assistant",
        instructions="You're a helpful WhatsApp assistant that can assist guests that are staying in our Paris AirBnb. Use your knowledge base to best respond to customer queries. If you don't know the answer, say simply that you cannot help with question and advice to contact the host directly. Be friendly and funny.",
        tools=[{"type": "file_search"}],
        model="gpt-4-1106-preview",
        tool_resources={ "file_search": {"file_id": [file.id]}}
)

BadRequestError: Error code: 400 - {'error': {'message': "Unknown parameter: 'tool_resources.file_search.file_id'.", 'type': 'invalid_request_error', 'param': 'tool_resources.file_search.file_id', 'code': 'unknown_parameter'}}

In [54]:
def create_assistant(file):
    """
    You currently cannot set the temperature for Assistant via the API.
    """
    assistant = client.beta.assistants.create(
        name="WhatsApp AirBnb Assistant",
        instructions="You're a helpful WhatsApp assistant that can assist guests that are staying in our Paris AirBnb. Use your knowledge base to best respond to customer queries. If you don't know the answer, say simply that you cannot help with question and advice to contact the host directly. Be friendly and funny.",
        tools=[{"type": "file_search"}],
        model="gpt-4-1106-preview",
        # tool_resources={ "file_search": {"file_ids": [file.id]}}
    )
    return assistant


In [55]:
assistant = create_assistant(file)
assistant

Assistant(id='asst_l33Nsitd0BPlbs4TmDRfnLlq', created_at=1730388597, description=None, instructions="You're a helpful WhatsApp assistant that can assist guests that are staying in our Paris AirBnb. Use your knowledge base to best respond to customer queries. If you don't know the answer, say simply that you cannot help with question and advice to contact the host directly. Be friendly and funny.", metadata={}, model='gpt-4-1106-preview', name='WhatsApp AirBnb Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

In [62]:
assistant.id

'asst_l33Nsitd0BPlbs4TmDRfnLlq'

## Thread management

In [57]:
def check_if_thread_exists(wa_id):
    with shelve.open("threads_db") as threads_shelf:
        return threads_shelf.get(wa_id, None)

In [58]:
def store_thread(wa_id, thread_id):
    with shelve.open("threads_db", writeback=True) as threads_shelf:
        threads_shelf[wa_id] = thread_id

## Generate response

In [59]:
def generate_response(message_body, wa_id, name):
    # Check if there is already a thread_id for the wa_id
    thread_id = check_if_thread_exists(wa_id)

    # If a thread doesn't exist, create one and store it
    if thread_id is None:
        print(f"Creating new thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.create()
        store_thread(wa_id, thread.id)
        thread_id = thread.id

    # Otherwise, retrieve the existing thread
    else:
        print(f"Retrieving existing thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.retrieve(thread_id)

    # Add message to thread
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
    )

    # Run the assistant and get the new message
    new_message = run_assistant(thread)
    print(f"To {name}:", new_message)
    return new_message

## Run assistant

In [65]:
def run_assistant(thread):
    # Retrieve the Assistant
    assistant = client.beta.assistants.retrieve('asst_l33Nsitd0BPlbs4TmDRfnLlq')

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # Wait for completion
    while run.status != "completed":
        # Be nice to the API
        time.sleep(0.5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve the Messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value
    print(f"Generated message: {new_message}")
    return new_message

## Test assistant

In [66]:
new_message = generate_response("What's the check in time?", "123", "John")
new_message

Retrieving existing thread for John with wa_id 123
Generated message: Oh la la, it seems we don't have the document with the check-in details just yet. Can you please provide me with the house manual or any information the host has shared with you? Once I have that, I'll be able to pinpoint the check-in time faster than you can say "baguette"! If you don't have these details, I'd recommend reaching out directly to the host to ensure a smooth arrival.
To John: Oh la la, it seems we don't have the document with the check-in details just yet. Can you please provide me with the house manual or any information the host has shared with you? Once I have that, I'll be able to pinpoint the check-in time faster than you can say "baguette"! If you don't have these details, I'd recommend reaching out directly to the host to ensure a smooth arrival.


'Oh la la, it seems we don\'t have the document with the check-in details just yet. Can you please provide me with the house manual or any information the host has shared with you? Once I have that, I\'ll be able to pinpoint the check-in time faster than you can say "baguette"! If you don\'t have these details, I\'d recommend reaching out directly to the host to ensure a smooth arrival.'

In [ ]:
new_message = generate_response("What's the pin for the lockbox?", "456", "Sarah")
new_message

In [ ]:
new_message = generate_response("What was my previous question?", "123", "John")
new_message

In [ ]:
new_message = generate_response("What was my previous question?", "456", "Sarah")
new_message